In [1]:
import pandas as pd
import numpy as np
import re 
import json
import ast
import nltk
from nltk.stem import WordNetLemmatizer
import google.generativeai as genai
import re
import neo4j
import spacy

/Users/ayush/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayush/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!python3 -m spacy download en_core_web_sm


In [2]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import pandas as pd
import ast
from py2neo import Graph, Node, Relationship

# ==== STEP 1: Load and Preprocess Data ====

# Load your dataset
df = pd.read_csv('/Users/ayush/Desktop/flipkart_neo4j/flipkart_com-ecommerce_sample.csv')  # Update with the correct file path

# Select only the required columns
filtered_df = df[['uniq_id', 'product_name', 'product_category_tree', 
                  'brand', 'product_specifications', 
                  'retail_price', 'discounted_price']].copy()

filtered_df.head()
df = filtered_df
df.head()



In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import re
from tqdm import tqdm  # Import tqdm for progress bar

# Neo4j Credentials
NEO4J_URI = "neo4j+s://be36b918.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "pvkX5XeO_sWXQqwE9BNDZkf8csnlADKfKho__evcabg"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Initialize counters
product_count = 0
relationship_count = 0

def sanitize_relationship_name(attribute):
    """Sanitizes attribute names to be valid Neo4j relationship names."""
    return re.sub(r'[^a-zA-Z0-9]', '_', attribute).upper()  # Replace special chars with '_'

def create_product_nodes(tx, product):
    query = """
    MERGE (p:Product {uniq_id: $uniq_id})
    SET p.product_name = $product_name, p.brand = $brand, p.retail_price = $retail_price
    """
    tx.run(query, **product)

def create_attribute_relationships(tx, product_id, attributes):
    global relationship_count
    for attribute, value in attributes.items():
        if pd.isna(value) or value is None or value == "":  # Skip NULL values
            continue  

        sanitized_attr = sanitize_relationship_name(attribute)  # Sanitize relationship name
        query = f"""
            MATCH (p:Product {{uniq_id: $uniq_id}})
            MERGE (a:Attribute {{name: $value}})
            MERGE (p)-[:HAS_{sanitized_attr}]->(a)
        """
        tx.run(query, uniq_id=product_id, value=value)  
        relationship_count += 1  # Increment relationship count

# Insert products into Neo4j with tqdm progress bar
with driver.session() as session:
    for _, row in tqdm(df_expanded.iterrows(), total=len(df_expanded), desc="Processing Rows", unit="row"):
        product_data = {
            "uniq_id": row["uniq_id"],
            "product_name": row["Product Name"],
            "brand": row["Brand"],
            "retail_price": row["Retail Price"]
        }
        session.write_transaction(create_product_nodes, product_data)
        product_count += 1  

        attributes = row.drop(["uniq_id", "Product Name", "Brand", "Retail Price"]).dropna().to_dict()
        session.write_transaction(create_attribute_relationships, row["uniq_id"], attributes)

# Close connection
driver.close()

# Print final statistics
print("\n✅ Knowledge Graph created successfully!")
print(f"🟢 Total Products Added: {product_count}")
print(f"🔵 Total Attribute Relationships Created: {relationship_count}")


In [3]:
import pandas as pd
import numpy as np
import re 
import json
import ast
import nltk
from nltk.stem import WordNetLemmatizer
import google.generativeai as genai
import re
import neo4j
import spacy
from neo4j import GraphDatabase
import pandas as pd
import re
from tqdm import tqdm  # Import tqdm for progress bar
from collections import defaultdict
import json


# Neo4j Credentials
NEO4J_URI = "neo4j+s://be36b918.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "pvkX5XeO_sWXQqwE9BNDZkf8csnlADKfKho__evcabg"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

lemmatizer = WordNetLemmatizer()

### --- Helper Functions ---
def lemmatize_word(word):
    """Convert a word to its root form (lemma)."""
    return lemmatizer.lemmatize(word.lower())

def preprocess_input(user_input):
    """Preprocess user input by lemmatizing words and removing stopwords."""
    stopwords = {"and", "or", "the", "of", "with", "for", "in"}
    words = re.split(r'\s+', user_input.lower())
    keywords = [lemmatize_word(word.strip()) for word in words if word not in stopwords]
    return keywords

def get_pos_tags(user_input):
    """Extract words and their POS tags from input."""
    doc = nlp(user_input)
    return {token.text.lower(): token.pos_ for token in doc}  # word → POS mapping

    
### --- Main Function ---
def fetch_products_by_lemmatized_attributes(user_input):
    """Fetch products that match lemmatized input attributes across all columns."""
    
    keywords = preprocess_input(user_input)
    if not keywords:
        return None

    query = """
    MATCH (p:Product)
    OPTIONAL MATCH (p)-[r]->(a)  
    WITH p, 
         COALESCE(a, {name: "N/A"}) AS a,  
         [key IN keys(p) | toLower(toString(p[key]))] AS product_values,  
         [key IN keys(a) | toLower(toString(a[key]))] AS attribute_values
    WITH p, attribute_values + product_values AS all_values, a
    WHERE any(keyword IN $keywords WHERE 
              any(value IN all_values WHERE value CONTAINS keyword))
    RETURN DISTINCT 
        p.uniq_id AS Product_ID, 
        p.product_name AS Product_Name, 
        p.brand AS Brand, 
        p.retail_price AS Price, 
        COLLECT(DISTINCT a.name) AS Matching_Attributes, 
        [keyword IN $keywords WHERE 
            any(value IN all_values WHERE value CONTAINS keyword)] AS Matched_Words
    """

    with driver.session() as session:
        results = session.run(query, keywords=keywords)
        products = [record.data() for record in results]

    if not products:
        print("\n❌ No matching products found.")
        return None

    df_results = pd.DataFrame(products)

    # ✅ Ensure "Matched_Words" exists before processing
    if "Matched_Words" not in df_results.columns or df_results.empty:
        print("\n⚠️ No valid Matched_Words found in results.")
        return None

    # ✅ Convert Matched_Words list to a readable string
    df_results["Matched_Words"] = df_results["Matched_Words"].apply(
        lambda x: ", ".join(x) if x else "N/A"
    )

    # ✅ Handle missing "Matched_Word" values safely
    df_results["Matched_Word"] = df_results["Matched_Words"].apply(
        lambda x: x.split(", ")[0] if x != "N/A" and x else "N/A"
    )

    return reorder_results(df_results, user_input)  # Apply POS-based ordering


### --- POS-based Sorting ---
def reorder_results(df_results, user_input):
    """Reorders DataFrame based on POS-based groups (Nouns → Other → Verbs → Adjectives)."""
    
    pos_tags = get_pos_tags(user_input)  # Get POS for words
    
    # ✅ Ensure "Matched_Word" column exists before processing
    if "Matched_Word" not in df_results.columns:
        print("\n⚠️ 'Matched_Word' column is missing.")
        return df_results

    # Group results by POS category
    grouped_results = defaultdict(list)
    for _, row in df_results.iterrows():
        match_word = row["Matched_Word"].lower() if isinstance(row["Matched_Word"], str) else "N/A"
        pos = pos_tags.get(match_word, "OTHER")  # Default to OTHER if POS not found
        grouped_results[pos].append(row)

    # Define priority order
    pos_order = ["NOUN", "OTHER", "VERB", "ADJ"]
    
    # Flatten results: All nouns → All other → All verbs → All adjectives
    reordered = []
    for pos in pos_order:
        if pos in grouped_results:
            reordered.extend(grouped_results[pos])

    return pd.DataFrame(reordered)


### --- Example Run ---
user_input = "pink saree"
df_results = fetch_products_by_lemmatized_attributes(user_input)
df_results = df_results[['Product_Name']]
json_results = json.dumps(json.loads(df_results.to_json(orient="records")), indent=4)
json_results

'[\n    {\n        "Product_Name": "Jayanti sarees Party Embroidered Women\'s Kurti"\n    },\n    {\n        "Product_Name": "JUHI CREATIONS GW2 Brooch"\n    },\n    {\n        "Product_Name": "Bunny Sarees Cotton Embroidered Blouse Material"\n    },\n    {\n        "Product_Name": "Sareeshop Silk Checkered Blouse Material"\n    },\n    {\n        "Product_Name": "dilli bazaaar Bellies, Corporate Casuals, Casuals"\n    },\n    {\n        "Product_Name": "Vishudh Printed Women\'s Straight Kurta"\n    },\n    {\n        "Product_Name": "Vishudh Printed Women\'s Anarkali Kurta"\n    },\n    {\n        "Product_Name": "Vishudh Printed Women\'s Straight Kurta"\n    },\n    {\n        "Product_Name": "Clincher Semonday Boots"\n    },\n    {\n        "Product_Name": "Masaba for Magic fairy Girl\'s Layered Pink Dress"\n    },\n    {\n        "Product_Name": "Serebroarts Iron Decorative Platter"\n    },\n    {\n        "Product_Name": "Pink Rose Alloy Bracelet"\n    },\n    {\n        "Product_

In [4]:
df_results

,Product_Name
7,Jayanti sarees Party Embroidered Women's Kurti
235,JUHI CREATIONS GW2 Brooch
273,Bunny Sarees Cotton Embroidered Blouse Material
286,Sareeshop Silk Checkered Blouse Material
0,"dilli bazaaar Bellies, Corporate Casuals, Casuals"
...,...
281,YugshaJewels Brass Necklace
282,Adorelabel Exquisite Gold and Pink Designer Me...
283,"Voylla Metal, Alloy Necklace"
284,Kandoi Cotton Silk Blend Embroidered Blouse Ma...


In [8]:
import json

json_results = json.dumps(json.loads(df_results.to_json(orient="records")), indent=4)
json_results

'[\n    {\n        "Product_ID": "2",\n        "Product_Name": "AW Bellies",\n        "Brand": "AW",\n        "Price": 999.0,\n        "Matching_Attributes": [\n            "Patent Leather"\n        ],\n        "Matched_Words": "leather",\n        "Matched_Word": "leather"\n    },\n    {\n        "Product_ID": "23",\n        "Product_Name": "SWAGGA Women Clogs",\n        "Brand": "SWAGGA",\n        "Price": 1500.0,\n        "Matching_Attributes": [\n            "Leather"\n        ],\n        "Matched_Words": "leather",\n        "Matched_Word": "leather"\n    },\n    {\n        "Product_ID": "36",\n        "Product_Name": "Packman 8 x 10 inches Security Bags Without POD Jacket Courier Bag Security Bag",\n        "Brand": "Packman",\n        "Price": 350.0,\n        "Matching_Attributes": [\n            "8 x 10 inches Security Bags Without POD Jacket Courier Bag",\n            "600 g",\n            "Grey",\n            "1 pack contains 100 pcs",\n            "Plastic",\n            "100"

In [ ]:
### **Step 2: Fetch Related Objects Using Gemini**
def get_ecommerce_related_objects_for_word(word):
    """
    Generate 3 most relevant single-word objects commonly sold on e-commerce platforms.
    """
    prompt = f"""
        Given the keyword: '{word}', generate exactly 3 relevant, single-word **tangible objects** that are explicitly mentioned or strongly implied.

        - Ensure each object is a **noun** (avoid adjectives, actions, or abstract concepts).  
        - Each object should be **one word only** (no multi-word phrases).  
        - Focus strictly on **physical, tangible products** commonly found in e-commerce listings.  
        - Return the objects in a **comma-separated list**.  

        Output format: object1, object2, object3
    """

    try:
        model = genai.GenerativeModel("gemini-1.5-pro")  
        response = model.generate_content([prompt])
        text = response.text.strip()

        words = text.split(",") if "," in text else text.split("\n")
        cleaned_words = [w.strip().split()[0] for w in words]  # Take only the first word
        return cleaned_words[:3]  # Limit to 3 objects per word
    except Exception as e:
        print(f"Error for word '{word}':", e)
        return []

def get_ecommerce_related_objects(sentence):
    """
    Generate a list of relevant objects for all words in the sentence.
    """
    filtered_words = preprocess_input(sentence)
    all_related_objects = []

    for word in filtered_words:
        all_related_objects.extend(get_ecommerce_related_objects_for_word(word))

    return all_related_objects